# 재난문자방송 발령현황 API Service Crawling
- api 데이터 수집
- 데이터 전처리
- sqlite db 생성

In [1]:
import requests
import json
from tqdm import tqdm
import pandas as pd

In [4]:
# 데이터 확인

url = 'http://apis.data.go.kr/1741000/DisasterMsg2/getDisasterMsgList?serviceKey=aTL0a3x%2ByeHsyPoG7zVzNPKayg5sxWKeU8Y53iusLP21UVu9hXOxtgYcS3Lhf7wes478p0zi2KD5JTdYZrP3ew%3D%3D&pageNo=1&numOfRows=10&type=json&flag=Y&'
resp = requests.get(url)
json = resp.json()
json

{'DisasterMsg': [{'head': [{'totalCount': 25190},
    {'numOfRows': '10', 'pageNo': '1', 'type': 'JSON'},
    {'RESULT': {'resultCode': 'INFO-0', 'resultMsg': '정상'}}]},
  {'row': [{'create_date': '2020/08/18 10:07:47',
     'location_id': '174',
     'location_name': '인천광역시 부평구',
     'md101_sn': '45692',
     'msg': '[부평구청]확진환자 4명 발생(성북구사랑제일교회관련/부평2동 1/산곡3동 2/십정1동 1).방역 및 역학조사중. blog.naver.com/withbupyeong',
     'send_platform': 'cbs'},
    {'create_date': '2020/08/18 10:07:27',
     'location_id': '82',
     'location_name': '경상북도 봉화군',
     'md101_sn': '45691',
     'msg': '[봉화군청]8.15(토) 서울 광복절 집회 및 서울 사랑제일교회 참석자는 증상유무와 관계없이 보건소로 상담바라며 수도권 등 타지역 방문을 자제바랍니다.',
     'send_platform': 'cbs'},
    {'create_date': '2020/08/18 10:07:14',
     'location_id': '184',
     'location_name': '전라남도 구례군',
     'md101_sn': '45690',
     'msg': '[구례군청] 현재 구례군에 폭염경보 발효중입니다. 수해복구중인 주민과 자원봉사자들께서는 물 충분히 마시기, 마스크 착용 등 안전에 유의하며 작업해주시기 바랍니다.',
     'send_platform': 'cbs'},
    {'create_date': '2020/08/18 

In [13]:
# Crawling

def api_crawling():
    url = 'http://apis.data.go.kr/1741000/DisasterMsg2/getDisasterMsgList?serviceKey=aTL0a3x%2ByeHsyPoG7zVzNPKayg5sxWKeU8Y53iusLP21UVu9hXOxtgYcS3Lhf7wes478p0zi2KD5JTdYZrP3ew%3D%3D&pageNo=1&numOfRows=10&type=json&flag=Y&'
    
    resp = requests.get(url)
    
    totalCount = resp.json()['DisasterMsg'][0]['head'][0]['totalCount']
    
    create_date = []
    location_id = []
    location_name = []
    md101_sn = []
    msg = []
    send_platform = []
    
    # totalCount가 numOfRows=1000으로 나누어떨어지는 경우
    if (totalCount % 1000) == 0:
        total_page = (totalCount // 1000) + 1
    # totalCount가 numOfRows=1000으로 나누어떨어지지 않는 경우
    else:
        total_page = (totalCount // 1000) + 2
        
    for pageNo in tqdm(range(1, total_page)):
        url = 'http://apis.data.go.kr/1741000/DisasterMsg2/getDisasterMsgList?serviceKey=aTL0a3x%2ByeHsyPoG7zVzNPKayg5sxWKeU8Y53iusLP21UVu9hXOxtgYcS3Lhf7wes478p0zi2KD5JTdYZrP3ew%3D%3D&numOfRows=1000&type=json&flag=Y&&pageNo='+str(pageNo)

        resp = requests.get(url)

        json_data = resp.json()

        row = json_data['DisasterMsg'][1]['row']

        for data in row:
            create_date.append(data['create_date'])
            location_id.append(data['location_id'])
            location_name.append(data['location_name'])
            md101_sn.append(data['md101_sn'])
            msg.append(data['msg'])
            send_platform.append(data['send_platform'])

    msg_api_df = pd.DataFrame({"create_date": create_date, "location_id": location_id, "location_name":location_name,
                               "md101_sn":md101_sn, "msg":msg, "send_platform":send_platform})

    # DataFrame 형식으로 반환
    return msg_api_df

In [15]:
msg_api_df = api_crawling()



  0%|                                                                       | 0/26 [00:00<?, ?it/s]

  4%|██▍                                                            | 1/26 [00:00<00:17,  1.47it/s]

  8%|████▊                                                          | 2/26 [00:01<00:16,  1.47it/s]

 12%|███████▎                                                       | 3/26 [00:02<00:15,  1.44it/s]

 15%|█████████▋                                                     | 4/26 [00:02<00:14,  1.47it/s]

 19%|████████████                                                   | 5/26 [00:03<00:14,  1.50it/s]

 23%|██████████████▌                                                | 6/26 [00:06<00:25,  1.26s/it]

 27%|████████████████▉                                              | 7/26 [00:06<00:20,  1.07s/it]

 31%|███████████████████▍                                           | 8/26 [00:07<00:16,  1.06it/s]

 35%|█████████████████████▊                                         | 9/26 [00:07<00:14, 

In [16]:
msg_api_df

,create_date,location_id,location_name,md101_sn,msg,send_platform
0,2020/08/18 10:07:47,174,인천광역시 부평구,45692,[부평구청]확진환자 4명 발생(성북구사랑제일교회관련/부평2동 1/산곡3동 2/십정1...,cbs
1,2020/08/18 10:07:27,82,경상북도 봉화군,45691,[봉화군청]8.15(토) 서울 광복절 집회 및 서울 사랑제일교회 참석자는 증상유무와...,cbs
2,2020/08/18 10:07:14,184,전라남도 구례군,45690,[구례군청] 현재 구례군에 폭염경보 발효중입니다. 수해복구중인 주민과 자원봉사자들께...,cbs
3,2020/08/18 10:06:27,86,경상북도 영덕군,45689,"[영덕군청] 관내 폭염경보 지속 발효중. 물 충분히 마시기, 무더위 쉼터 이용, 실...",cbs
4,2020/08/18 10:02:57,127,부산광역시 북구,45688,[북구청]북구 코로나19 감염증 추가확진자 1명 발생. 감염경로 및 동선 파악중. ...,cbs
...,...,...,...,...,...,...
25185,2011/12/08 21:57:06,"3,4,5,6,7,9,13,16","강원도 강릉시,강원도 고성군,강원도 동해시,강원도 삼척시,강원도 속초시,강원도 양양...",33,[국민안전처]\n12.8 21:40 강원(강릉평지 동해산간 태백 삼척산간 속초평지 ...,cbs
25186,2011/12/08 19:46:08,"3,4,5,6,7,9,12,13,17","강원도 강릉시,강원도 고성군,강원도 동해시,강원도 삼척시,강원도 속초시,강원도 양양...",32,"[강릉국도]\n현재 영동지방 많은눈으로 인해 전면통제(진부령,진고개), 부분통제(삽...",cbs
25187,2011/12/08 19:34:21,"3,17,18","강원도 강릉시,강원도 평창군,강원도 홍천군",31,"[국민안전처]\n12.8 19:30 강원산간(강릉 평창 홍천) 대설경보대치, 비닐하...",cbs
25188,2011/12/08 18:36:01,"4,7,9,12","강원도 고성군,강원도 속초시,강원도 양양군,강원도 인제군",30,"[국민안전처]\n12.8 18:30 강원산간(속초 고성 양양 인제)지역 대설경보,비...",cbs


# 재난문자 전처리
- 호우 관련 재난문자 및 수도권 내의 재난문자
- 수도권을 경기, 서울, 인천 기준으로 한 이유는 교량 및 수위 관측소가 해당 지역 밖에 없기 때문에

In [17]:
msg_api_df = msg_api_df.drop('send_platform', axis=1)

In [18]:
msg_api_df

,create_date,location_id,location_name,md101_sn,msg
0,2020/08/18 10:07:47,174,인천광역시 부평구,45692,[부평구청]확진환자 4명 발생(성북구사랑제일교회관련/부평2동 1/산곡3동 2/십정1...
1,2020/08/18 10:07:27,82,경상북도 봉화군,45691,[봉화군청]8.15(토) 서울 광복절 집회 및 서울 사랑제일교회 참석자는 증상유무와...
2,2020/08/18 10:07:14,184,전라남도 구례군,45690,[구례군청] 현재 구례군에 폭염경보 발효중입니다. 수해복구중인 주민과 자원봉사자들께...
3,2020/08/18 10:06:27,86,경상북도 영덕군,45689,"[영덕군청] 관내 폭염경보 지속 발효중. 물 충분히 마시기, 무더위 쉼터 이용, 실..."
4,2020/08/18 10:02:57,127,부산광역시 북구,45688,[북구청]북구 코로나19 감염증 추가확진자 1명 발생. 감염경로 및 동선 파악중. ...
...,...,...,...,...,...
25185,2011/12/08 21:57:06,"3,4,5,6,7,9,13,16","강원도 강릉시,강원도 고성군,강원도 동해시,강원도 삼척시,강원도 속초시,강원도 양양...",33,[국민안전처]\n12.8 21:40 강원(강릉평지 동해산간 태백 삼척산간 속초평지 ...
25186,2011/12/08 19:46:08,"3,4,5,6,7,9,12,13,17","강원도 강릉시,강원도 고성군,강원도 동해시,강원도 삼척시,강원도 속초시,강원도 양양...",32,"[강릉국도]\n현재 영동지방 많은눈으로 인해 전면통제(진부령,진고개), 부분통제(삽..."
25187,2011/12/08 19:34:21,"3,17,18","강원도 강릉시,강원도 평창군,강원도 홍천군",31,"[국민안전처]\n12.8 19:30 강원산간(강릉 평창 홍천) 대설경보대치, 비닐하..."
25188,2011/12/08 18:36:01,"4,7,9,12","강원도 고성군,강원도 속초시,강원도 양양군,강원도 인제군",30,"[국민안전처]\n12.8 18:30 강원산간(속초 고성 양양 인제)지역 대설경보,비..."


In [25]:
rain_msg_df = msg_api_df[msg_api_df["msg"].str.contains("호우")] # 호우 관련 재난문자만 전처리

In [23]:
pd.unique(msg_api_df['location_name']).tolist()

['인천광역시 부평구',
 '경상북도 봉화군',
 '전라남도 구례군',
 '경상북도 영덕군',
 '부산광역시 북구',
 '서울특별시 금천구',
 '세종특별자치시 ',
 '경상북도 전체',
 '강원도 원주시',
 '강원도 강릉시',
 '인천광역시 서구',
 '충청북도 음성군',
 '인천광역시 전체',
 '경상북도 포항시',
 '충청남도 서천군',
 '경상북도 영주시',
 '전라북도 군산시',
 '전라남도 완도군',
 '경상북도 안동시',
 '인천광역시 남동구',
 '경기도 가평군',
 '경기도 안산시',
 '전라북도 남원시',
 '경상북도 칠곡군',
 '서울특별시 양천구',
 '전라북도 완주군',
 '전라남도 장흥군',
 '전라남도 순천시',
 '강원도 춘천시',
 '강원도 동해시,강원도 삼척시,강원도 정선군,강원도 홍천군,강원도 횡성군,충청남도 보령시',
 '경기도 양평군',
 '경기도 오산시',
 '전라북도 김제시',
 '충청남도 천안시',
 '충청북도 청주시',
 '전라북도 고창군',
 '전라남도 영광군',
 '경상남도 진주시',
 '광주광역시 전체',
 '전라남도 광양시',
 '경상북도 문경시',
 '경상북도 예천군',
 '경기도 수원시',
 '경기도 화성시',
 '경기도 용인시',
 '서울특별시 도봉구',
 '서울특별시 강동구',
 '서울특별시 노원구',
 '경상남도 고성군',
 '전라북도 전주시',
 '전라북도 정읍시',
 '서울특별시 중랑구',
 '경기도 광명시',
 '서울특별시 성북구',
 '서울특별시 동작구',
 '서울특별시 광진구',
 '경기도 양주시',
 '서울특별시 동대문구',
 '경기도 성남시',
 '경기도 남양주시',
 '충청남도 공주시',
 '경기도 동두천시',
 '전라남도 여수시',
 '경상남도 통영시',
 '강원도 평창군',
 '전라남도 진도군',
 '서울특별시 영등포구',
 '경기도 안양시',
 '울산광역시 전체',
 '충청남도 당진시',
 '전라남도 전체',
 '부산광역시 남구',
 '충청남도 홍성군',
 '경기도 평택시',
 

In [ ]:
msg_api_df.isna().sum()

In [28]:
sudo_rain_df = rain_msg_df[rain_msg_df['location_name'].str.contains("경기" or "서울" or "인천")]
sudo_rain_df

,create_date,location_id,location_name,md101_sn,msg
1084,2020/08/15 10:54:01,49,경기도 평택시,44608,"[평택시청] 최근 수도권에서 코로나19 감염이 급속도로 확산되고 있으니 종교모임, ..."
1122,2020/08/15 09:36:34,22,경기도 가평군,44570,"[가평군청] 집중호우에 따른 산사태, 저지대 침수, 급류휩쓸림, 도로유실 등 위험사..."
1126,2020/08/15 09:31:47,41,경기도 여주시,44566,"[여주시청]15일(토) 여주시 호우주의보 발효중. 저지대 침수 우려지역, 특히 산사..."
1144,2020/08/15 08:12:58,44,경기도 용인시,44548,[용인시청] 용인시 호우주의보 발효 중. 산 인근 방문자제 및 산사태·상습침수 등 ...
1161,2020/08/14 21:17:35,45,경기도 의왕시,44531,"[의왕시청] 내일 새벽부터 아침까지 집중호우 예상. 하천변, 저지대, 산사태 위험지..."
...,...,...,...,...,...
25094,2012/07/06 01:51:24,"23,24,25,29,32,136","경기도 고양시,경기도 과천시,경기도 광명시,경기도 김포시,경기도 부천시,서울특별시 전체",243,"[국민안전처]7.6일01시40분 광명,과천,부천,김포,고양,서울 호우경보, 상습침수..."
25095,2012/07/06 01:40:33,"33,37,38,43,169","경기도 성남시,경기도 안성시,경기도 안양시,경기도 오산시,인천광역시 강화군",242,"[국민안전처]\n7.6일 01시30분발효 강화,성남, 안양, 오산, 안성 지역 호우..."
25096,2012/07/06 01:03:10,"26,41,44,47,49,177","경기도 광주시,경기도 여주시,경기도 용인시,경기도 이천시,경기도 평택시,인천광역시 옹진군",241,"[국민안전처]6일0시50분 평택,용인,이천,여주,광주,서해5도지역 호우경보,상습침수..."
25097,2012/07/06 00:38:33,"28,34,45","경기도 군포시,경기도 수원시,경기도 의왕시",240,"[국민안전처]\n6일00시30분 수원시, 군포시, 의왕시지역 호우경보, 상습침수및위..."


In [32]:
sudo_rain_df = sudo_rain_df.reset_index()
sudo_rain_df

,index,create_date,location_id,location_name,md101_sn,msg
0,1084,2020/08/15 10:54:01,49,경기도 평택시,44608,"[평택시청] 최근 수도권에서 코로나19 감염이 급속도로 확산되고 있으니 종교모임, ..."
1,1122,2020/08/15 09:36:34,22,경기도 가평군,44570,"[가평군청] 집중호우에 따른 산사태, 저지대 침수, 급류휩쓸림, 도로유실 등 위험사..."
2,1126,2020/08/15 09:31:47,41,경기도 여주시,44566,"[여주시청]15일(토) 여주시 호우주의보 발효중. 저지대 침수 우려지역, 특히 산사..."
3,1144,2020/08/15 08:12:58,44,경기도 용인시,44548,[용인시청] 용인시 호우주의보 발효 중. 산 인근 방문자제 및 산사태·상습침수 등 ...
4,1161,2020/08/14 21:17:35,45,경기도 의왕시,44531,"[의왕시청] 내일 새벽부터 아침까지 집중호우 예상. 하천변, 저지대, 산사태 위험지..."
...,...,...,...,...,...,...
281,25094,2012/07/06 01:51:24,"23,24,25,29,32,136","경기도 고양시,경기도 과천시,경기도 광명시,경기도 김포시,경기도 부천시,서울특별시 전체",243,"[국민안전처]7.6일01시40분 광명,과천,부천,김포,고양,서울 호우경보, 상습침수..."
282,25095,2012/07/06 01:40:33,"33,37,38,43,169","경기도 성남시,경기도 안성시,경기도 안양시,경기도 오산시,인천광역시 강화군",242,"[국민안전처]\n7.6일 01시30분발효 강화,성남, 안양, 오산, 안성 지역 호우..."
283,25096,2012/07/06 01:03:10,"26,41,44,47,49,177","경기도 광주시,경기도 여주시,경기도 용인시,경기도 이천시,경기도 평택시,인천광역시 옹진군",241,"[국민안전처]6일0시50분 평택,용인,이천,여주,광주,서해5도지역 호우경보,상습침수..."
284,25097,2012/07/06 00:38:33,"28,34,45","경기도 군포시,경기도 수원시,경기도 의왕시",240,"[국민안전처]\n6일00시30분 수원시, 군포시, 의왕시지역 호우경보, 상습침수및위..."


In [37]:
# 엑셀 파일 저장
sudo_rain_df.to_csv('호우_재난문자.csv')

# sqlite DB 만들기
- 서울, 인천, 경기별로 재난문자 select 후 map에 시각화
- 실시간으로 수집한 수도권 호우주의 재난문자는 DB에 저장되도록 코드 작성(셀레니움 활용한 realtime_재난문자크롤러.ipynb 코드 수정 필요)

In [35]:
import sqlite3 as sql

In [38]:
# DB로 저장
con = sql.connect('rain_msg.db')
sudo_rain_df.to_sql('RAIN_MSG', con, if_exists='append', index=False)
con.close()

In [39]:
# DB 불러오기
con = sql.connect('rain_msg.db')
df = pd.read_sql('SELECT * FROM RAIN_MSG', con)
df

,index,create_date,location_id,location_name,md101_sn,msg
0,1084,2020/08/15 10:54:01,49,경기도 평택시,44608,"[평택시청] 최근 수도권에서 코로나19 감염이 급속도로 확산되고 있으니 종교모임, ..."
1,1122,2020/08/15 09:36:34,22,경기도 가평군,44570,"[가평군청] 집중호우에 따른 산사태, 저지대 침수, 급류휩쓸림, 도로유실 등 위험사..."
2,1126,2020/08/15 09:31:47,41,경기도 여주시,44566,"[여주시청]15일(토) 여주시 호우주의보 발효중. 저지대 침수 우려지역, 특히 산사..."
3,1144,2020/08/15 08:12:58,44,경기도 용인시,44548,[용인시청] 용인시 호우주의보 발효 중. 산 인근 방문자제 및 산사태·상습침수 등 ...
4,1161,2020/08/14 21:17:35,45,경기도 의왕시,44531,"[의왕시청] 내일 새벽부터 아침까지 집중호우 예상. 하천변, 저지대, 산사태 위험지..."
...,...,...,...,...,...,...
281,25094,2012/07/06 01:51:24,"23,24,25,29,32,136","경기도 고양시,경기도 과천시,경기도 광명시,경기도 김포시,경기도 부천시,서울특별시 전체",243,"[국민안전처]7.6일01시40분 광명,과천,부천,김포,고양,서울 호우경보, 상습침수..."
282,25095,2012/07/06 01:40:33,"33,37,38,43,169","경기도 성남시,경기도 안성시,경기도 안양시,경기도 오산시,인천광역시 강화군",242,"[국민안전처]\n7.6일 01시30분발효 강화,성남, 안양, 오산, 안성 지역 호우..."
283,25096,2012/07/06 01:03:10,"26,41,44,47,49,177","경기도 광주시,경기도 여주시,경기도 용인시,경기도 이천시,경기도 평택시,인천광역시 옹진군",241,"[국민안전처]6일0시50분 평택,용인,이천,여주,광주,서해5도지역 호우경보,상습침수..."
284,25097,2012/07/06 00:38:33,"28,34,45","경기도 군포시,경기도 수원시,경기도 의왕시",240,"[국민안전처]\n6일00시30분 수원시, 군포시, 의왕시지역 호우경보, 상습침수및위..."


In [46]:
# 경기
gyeong_df = pd.read_sql("SELECT * FROM RAIN_MSG WHERE location_name LIKE '%경기%'", con)
gyeong_df

,index,create_date,location_id,location_name,md101_sn,msg
0,1084,2020/08/15 10:54:01,49,경기도 평택시,44608,"[평택시청] 최근 수도권에서 코로나19 감염이 급속도로 확산되고 있으니 종교모임, ..."
1,1122,2020/08/15 09:36:34,22,경기도 가평군,44570,"[가평군청] 집중호우에 따른 산사태, 저지대 침수, 급류휩쓸림, 도로유실 등 위험사..."
2,1126,2020/08/15 09:31:47,41,경기도 여주시,44566,"[여주시청]15일(토) 여주시 호우주의보 발효중. 저지대 침수 우려지역, 특히 산사..."
3,1144,2020/08/15 08:12:58,44,경기도 용인시,44548,[용인시청] 용인시 호우주의보 발효 중. 산 인근 방문자제 및 산사태·상습침수 등 ...
4,1161,2020/08/14 21:17:35,45,경기도 의왕시,44531,"[의왕시청] 내일 새벽부터 아침까지 집중호우 예상. 하천변, 저지대, 산사태 위험지..."
...,...,...,...,...,...,...
281,25094,2012/07/06 01:51:24,"23,24,25,29,32,136","경기도 고양시,경기도 과천시,경기도 광명시,경기도 김포시,경기도 부천시,서울특별시 전체",243,"[국민안전처]7.6일01시40분 광명,과천,부천,김포,고양,서울 호우경보, 상습침수..."
282,25095,2012/07/06 01:40:33,"33,37,38,43,169","경기도 성남시,경기도 안성시,경기도 안양시,경기도 오산시,인천광역시 강화군",242,"[국민안전처]\n7.6일 01시30분발효 강화,성남, 안양, 오산, 안성 지역 호우..."
283,25096,2012/07/06 01:03:10,"26,41,44,47,49,177","경기도 광주시,경기도 여주시,경기도 용인시,경기도 이천시,경기도 평택시,인천광역시 옹진군",241,"[국민안전처]6일0시50분 평택,용인,이천,여주,광주,서해5도지역 호우경보,상습침수..."
284,25097,2012/07/06 00:38:33,"28,34,45","경기도 군포시,경기도 수원시,경기도 의왕시",240,"[국민안전처]\n6일00시30분 수원시, 군포시, 의왕시지역 호우경보, 상습침수및위..."


In [45]:
# 서울
seoul_df = pd.read_sql("SELECT * FROM RAIN_MSG WHERE location_name LIKE '%서울%'", con)
seoul_df

,index,create_date,location_id,location_name,md101_sn,msg
0,1697,2020/08/11 10:20:31,"2,21,53,74,113,136,168,202,217,222,238,6474","강원도 전체,경기도 전체,경상남도 전체,경상북도 전체,대전광역시 전체,서울특별시 전...",43995,"[중앙재난안전대책본부] 집중호우 위험, 저지대 침수, 비탈면 붕괴 등 위험지역에서 ..."
1,2143,2020/08/09 10:42:00,"8,10,11,12,13,14,15,16,17,18,19,20,21,113,136,...","강원도 양구군,강원도 영월군,강원도 원주시,강원도 인제군,강원도 정선군,강원도 철원...",43551,"[중앙재난안전대책본부]집중호우 위험, 저지대 침수, 비탈면 붕괴 등 위험지역에서 대..."
2,2250,2020/08/08 21:39:22,"22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,3...","경기도 가평군,경기도 고양시,경기도 과천시,경기도 광명시,경기도 광주시,경기도 구리...",43444,"[행정안전부] 오늘 22시30분 호우경보, 산사태ㆍ상습침수 등 위험지역 대피, 외출..."
3,3122,2020/08/06 10:08:04,"2,21,74,113,136,168,202,222,238,6474","강원도 전체,경기도 전체,경상북도 전체,대전광역시 전체,서울특별시 전체,인천광역시 ...",42572,"[중앙재난안전대책본부] 집중호우 위험, 저지대 침수, 비탈면 붕괴 등 위험지역에서 ..."
4,3209,2020/08/05 22:34:43,"2,21,53,74,98,104,113,119,136,162,168,179,202,...","강원도 전체,경기도 전체,경상남도 전체,경상북도 전체,광주광역시 전체,대구광역시 전...",42485,[산림청] 밤사이 많은 호우가 예보되어 산사태 발생 가능성 높음 산림 방문을 자제하...
5,3782,2020/08/02 12:01:52,"23,24,25,28,29,32,33,34,35,36,38,39,42,43,45,4...","경기도 고양시,경기도 과천시,경기도 광명시,경기도 군포시,경기도 김포시,경기도 부천...",41913,"[행정안전부] 오늘 14시00분 호우경보, 산사태ㆍ상습침수 등 위험지역 대피, 외출..."
6,21024,2019/09/10 21:30:21,"32,136","경기도 부천시,서울특별시 전체",24679,"[행정안전부] 오늘 21시30분 서울, 경기(부천),호우경보, 산사태ㆍ상습침수 등 ..."
7,21081,2019/09/05 14:55:07,"33,34,36,39,50,52,136","경기도 성남시,경기도 수원시,경기도 안산시,경기도 양주시,경기도 포천시,경기도 화성...",24622,"[행정안전부] 오늘 14시50분 경기(화성,양주,안산,수원,포천,성남),서울 호우경..."
8,21172,2019/07/26 07:16:50,"22,24,25,26,27,28,30,32,33,34,35,36,38,40,44,4...","경기도 가평군,경기도 과천시,경기도 광명시,경기도 광주시,경기도 구리시,경기도 군포...",24531,"[행정안전부]오늘 07시10분 서울,인천, 경기일부 호우경보,산사태ㆍ상습침수 등 위..."
9,21815,2018/08/29 18:45:11,"30,136,169,170,172,173,174,175,176,178","경기도 남양주시,서울특별시 전체,인천광역시 강화군,인천광역시 계양구,인천광역시 남동...",23886,"[행정안전부] 오늘 18시40분 인천(옹진제외), 서울, 경기(남양주) 호우경보, ..."


In [47]:
# 인천
incheon_df = pd.read_sql("SELECT * FROM RAIN_MSG WHERE location_name LIKE '%인천%'", con)
incheon_df

,index,create_date,location_id,location_name,md101_sn,msg
0,1697,2020/08/11 10:20:31,"2,21,53,74,113,136,168,202,217,222,238,6474","강원도 전체,경기도 전체,경상남도 전체,경상북도 전체,대전광역시 전체,서울특별시 전...",43995,"[중앙재난안전대책본부] 집중호우 위험, 저지대 침수, 비탈면 붕괴 등 위험지역에서 ..."
1,2143,2020/08/09 10:42:00,"8,10,11,12,13,14,15,16,17,18,19,20,21,113,136,...","강원도 양구군,강원도 영월군,강원도 원주시,강원도 인제군,강원도 정선군,강원도 철원...",43551,"[중앙재난안전대책본부]집중호우 위험, 저지대 침수, 비탈면 붕괴 등 위험지역에서 대..."
2,2250,2020/08/08 21:39:22,"22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,3...","경기도 가평군,경기도 고양시,경기도 과천시,경기도 광명시,경기도 광주시,경기도 구리...",43444,"[행정안전부] 오늘 22시30분 호우경보, 산사태ㆍ상습침수 등 위험지역 대피, 외출..."
3,3122,2020/08/06 10:08:04,"2,21,74,113,136,168,202,222,238,6474","강원도 전체,경기도 전체,경상북도 전체,대전광역시 전체,서울특별시 전체,인천광역시 ...",42572,"[중앙재난안전대책본부] 집중호우 위험, 저지대 침수, 비탈면 붕괴 등 위험지역에서 ..."
4,3209,2020/08/05 22:34:43,"2,21,53,74,98,104,113,119,136,162,168,179,202,...","강원도 전체,경기도 전체,경상남도 전체,경상북도 전체,광주광역시 전체,대구광역시 전...",42485,[산림청] 밤사이 많은 호우가 예보되어 산사태 발생 가능성 높음 산림 방문을 자제하...
5,3782,2020/08/02 12:01:52,"23,24,25,28,29,32,33,34,35,36,38,39,42,43,45,4...","경기도 고양시,경기도 과천시,경기도 광명시,경기도 군포시,경기도 김포시,경기도 부천...",41913,"[행정안전부] 오늘 14시00분 호우경보, 산사태ㆍ상습침수 등 위험지역 대피, 외출..."
6,21028,2019/09/10 17:48:40,"23,29,48,169,170,172,173,174,175,176,178","경기도 고양시,경기도 김포시,경기도 파주시,인천광역시 강화군,인천광역시 계양구,인천...",24675,"[행정안전부] 오늘 17시45분 경기(김포,고양,파주),인천(옹진 제외) 호우경보,..."
7,21172,2019/07/26 07:16:50,"22,24,25,26,27,28,30,32,33,34,35,36,38,40,44,4...","경기도 가평군,경기도 과천시,경기도 광명시,경기도 광주시,경기도 구리시,경기도 군포...",24531,"[행정안전부]오늘 07시10분 서울,인천, 경기일부 호우경보,산사태ㆍ상습침수 등 위..."
8,21176,2019/07/26 05:40:25,"14,23,29,31,39,48,50,169","강원도 철원군,경기도 고양시,경기도 김포시,경기도 동두천시,경기도 양주시,경기도 파...",24527,"[행정안전부]오늘 05시30분 경기북부,강화,철원 호우경보, 산사태ㆍ상습침수 등 위..."
9,21815,2018/08/29 18:45:11,"30,136,169,170,172,173,174,175,176,178","경기도 남양주시,서울특별시 전체,인천광역시 강화군,인천광역시 계양구,인천광역시 남동...",23886,"[행정안전부] 오늘 18시40분 인천(옹진제외), 서울, 경기(남양주) 호우경보, ..."
